# OpenStreetMap Data Wrangling Project

In this notebook I summarize my steps and findings towards the Data Wrangling Project course, using OpenStreet Map as a Case Study.
This is in the Scope of the Udacity Nanodegree, P3: Wrangle OpenStreetMap Data

** Author **: [Marcelo Tyszler](mailto:tyszler.jobs@gmail.com)

** Start Date **: 04/Mar/2017

** Today's Date **: 09/May/2017

## Choice of tools

For this project I opted for:

* Keep basic documentation and notes in the Jupyter Notebook
* Save files in Github
* Use PyCharm as Python environment
* Use DB Browser for SQLite as SQL environment (in combination with command line) for testing and exploration
* Having all SQL operations done via python using the sqlite API



## Map Area

Nieuwegein, Netherlands:

* Mapzen extract: https://mapzen.com/data/metro-extracts/your-extracts/2ac3c83d0cf8
* OpenStreetMap link: https://www.openstreetmap.org/relation/419212#map=12/52.0300/5.0966

I chose this area since this is where I currently live, and not a major city in the Netherlands. Therefore I believe I can collaborate better here due to my knowledge and have a marginal higher impact than to a major city like Amsterdam.

 *I downloaded the OSM XML option: https://s3.amazonaws.com/mapzen.odes/ex_4nypWySr8p3tPS5D3gfvy2aPey7mV.osm.bz2*


## Initial exploration

After loading a small sample of the data (taking every 10th entry in the full data), I explored the database.

I noticed the (initial) issues:

* **ISSUE 1**: Many unique keys: for example, *postal_code* and _postcode_ should be combined
* **ISSUE 2**: Post codes where not uniform (for example "3431 LN" and "3523ED")
* **ISSUE 3**: Phone numbers where not uniform

###  **ISSUE 1**: Many unique keys

#### Problem
I ran the following query:

This returned 134 unique rows, which I found suspiciously high. I did not investigate all details, but I did compare the top returning with the lower returning keys. Here I noticed, for example, that *postal_code* appeared with count 2, as opposed to *postcode* with count 5614.

#### Solution
I decided to merge postal_coe and postcode in the python import code:

###  **ISSUE 2**: Postcodes incomplete and not uniform

#### Problem
Browsing through the data revealed 2 problems with postcodes:
* Not uniform on the white space between 4 numbers and letters
* Incomplete postcodes having only 4 numbers

This can be seen in the following excerpt:

| value   |
|---------|
| 3432    |
| 3438    |
| 3437 PJ |
| 3433 ZB |
| 3431 LN |
| 3523ED  |
| 3523EJ  |
| 3523ED  |
| 3522HR  |
| 3438AA  |

#### Solution
The solution was implemented in 2 parts.

##### Part 1
First removed any whites spaces by adding the codes into the python db_prep.py code:

In [6]:
def improve_postcode(postcode):
    return postcode.replace(" ", "")

##### Part 2
To improve the postcodes with 4 digits only, I decided to use the information in the database to improve itself.

The approach was to find the closest node point with full postcode that started with the same 4 letters and use that full postcode as replacement.

This was done by combining the following subqueries:

| problem_postcode | key      | lat        | lon       |
|------------------|----------|------------|-----------|
| 3432             | postcode | 52.0101421 | 5.0832539 |
| 3438             | postcode | 52.0452865 | 5.101346  |
| 3439             | postcode | 52.0398723 | 5.1011803 |

| match_postcode | full_postcode | key      | lat        | lon       |
|----------------|---------------|----------|------------|-----------|
| 3437           | 3437PJ        | postcode | 52.0601311 | 5.0773771 |
| 3433           | 3433ZB        | postcode | 52.0040012 | 5.0958811 |
| 3431           | 3431LN        | postcode | 52.0281712 | 5.0807074 |
| 3523           | 3523ED        | postcode | 52.0721184 | 5.1245909 |
| 3523           | 3523EJ        | postcode | 52.072329  | 5.123319  |
| 3523           | 3523ED        | postcode | 52.0721409 | 5.1244553 |
| 3522           | 3522HR        | postcode | 52.0662376 | 5.114512  |
| 3438           | 3438AA        | postcode | 52.0377198 | 5.0928945 |
| 3526           | 3526KV        | postcode | 52.0616652 | 5.1101707 |
| 3431           | 3431HC        | postcode | 52.0276385 | 5.0827466 |

And after matching on the (filtered) postcode, to compute squared distances and find the minimum distance

| id         | problem_postcode | full_postcode | min(distance)        |
|------------|------------------|---------------|----------------------|
| 44844351   | 3432             | 3432ND        | 5.33922370001306e-07 |
| 45003711   | 3438             | 3438AM        | 8.83898600042912e-08 |
| 4352876693 | 3439             | 3439NK        | 1.35280450000897e-07 |

This was fed into the python create_db code,updating the problem postcode with the closest match

###  **ISSUE 3**: Phone numbers not uniform

#### Problem
I ran the following query:

| value             |
|-------------------|
| 030-6041144       |
| +31 (0)30 63 80   |
| 030-6056479       |
| +31-30-2804145    |
| +31-30-6006339    |
| +31302803890      |
| +31-30-6090821    |
| +31 (0) 302887760 |

The above excerpt reveals that there are the following inconsistencies in the way phone numbers are inserted:
* Inclusion or absence of the Dutch international code (+31)
* Spacing in between the numbers
* Use or not of hyphen "-" to separate regional code
* Insertion or not of (0) which is a version for local calling

#### Solution

I decided to reshape all phone numbers in the mask: +31 YY XXX XXXX, where:
* +31 is the Dutch international code
* YY is the regional code, which in this case will be typically 30
* XXX XXXX is the phone number, a combination of 3 + 4 numerical characters
* space is used for readability

This implemented by creating the auxiliar following python code into the db_prep.py:

In [2]:
def improve_phonenumbers(phonenumber):
    # remove all special chars:
    print phonenumber
    clean_number = phonenumber
    # remove (0)
    clean_number =  clean_number.replace("(0)","")
    # remove all non digits
    clean_number = re.sub(r'\D', "", clean_number)

    if len(clean_number) == 11:
        # full phone number
        formatted_number =  '+{0} {1} {2}-{3}'.format(clean_number[0:2], clean_number[2:4], clean_number[4:7], clean_number[7:])
    elif len(clean_number) == 10:
        # partial phone number, no international code, starting with 0
        formatted_number =  '+31 {0} {1}-{2}'.format(clean_number[1:3], clean_number[3:6], clean_number[6:])
    else:
        # invalid number
        formatted_number = ""

    return formatted_number


After the code, the same query returned

| value             |
|-------------------|
| +31 30 604-1144 |
| _NULL_                  |
| +31 30 605-6479 |
| +31 30 280-4145 |
| +31 30 600-6339 |
| +31 30 280-3890 |
| +31 30 609-0821 |
| +31 30 288-7760 |

## Further Exploration


After the initial exploration, further exploration was done with the full dataset

## Verify Cities
I was curious to see which cities would be listed. I would expect Nieuwegein, but also the neighbouring areas of Utrecht, Vianen, Ijsselstein.


| City            | Count |
|-----------------|-------|
| De Meern        | 44    |
| Hagestein       | 12    |
| Houten          | 24    |
| IJsselstein     | 7673  |
| Lopikerkapel    | 149   |
| Nieuwegein      | 31427 |
| Tull en 't Waal | 12    |
| Utrecht         | 14398 |
| Vianen          | 3094  |


The above result is fine and shows other small cities on the neighbourhood

## Other fields

Further inspection of the other fields did not reveal any (serious) problems. Street names were correct and did not suffer the abbreviation problems from typical US data.

# Data Overview

## File sizes

## Number of nodes

## Number of nodes_tags

## Number of ways

## Number of ways_tags

## Number of ways_nodes

## Number of unique users

### Top 10:
| User              | Contributions |
|-------------------|---------------|
| Martin Borsje_BAG | 193759        |
| Zugführer_BAG     | 47604         |
| 3dShapes          | 45710         |
| Hendrikklaas      | 41691         |
| rivw_BAG          | 38711         |
| Gertjan Idema_BAG | 36189         |
| ruudblank_BAG     | 19041         |
| Christoph Lotz    | 13181         |
| ligfietser        | 12619         |
| cartinus          | 6849          |

## Top 10 Amenities:

| Amenitiy        | counter |
|-----------------|---------|
| waste_basket    | 501     |
| bench           | 216     |
| parking         | 138     |
| recycling       | 64      |
| post_box        | 63      |
| bicycle_parking | 48      |
| fast_food       | 29      |
| restaurant      | 25      |
| school          | 24      |
| waste_disposal  | 20      |

# Other ideas:

* Further improvement can be done by cross check externally the postcodes
* Certain tags could be re-written to a more intuitively format. For example these tags refer to a cell tower:
		<tag k="height" v="21" />
		<tag k="man_made" v="tower" />
		<tag k="technology" v="UMTS" />
		<tag k="tower:type" v="communication" />
        
    It would be clearer to have all these 4 entries with a tag type of tower, instead of 'regular' and just one as 'tower', and the values as in there

      